This is an experimental (stub...) Schema.org dashboard using Jupyter/iPython

In [1]:
# My eyes...

import unittest
import os
from os import path, getenv
from os.path import expanduser
import logging # https://docs.python.org/2/library/logging.html#logging-levels
import glob
import argparse
import StringIO
import sys
sys.path.append( os.getcwd() ) 
sys.path.append( os.path.abspath(os.path.join("../", os.pardir))) 
sys.path.insert( 1, '../' ) 
sys.path.insert( 1, '../lib' ) #Pickup libs, rdflib etc., from shipped lib directory
sdk_path = getenv('APP_ENGINE',  expanduser("~") + '/google-cloud-sdk/platform/google_appengine/')
sys.path.insert(0, sdk_path) # add AppEngine SDK to path
import dev_appserver
dev_appserver.fix_sys_path()
from api import *
import rdflib
from rdflib import Graph
from rdflib import RDF, RDFS
from rdflib.term import URIRef, Literal
from rdflib.parser import Parser
from rdflib.serializer import Serializer
from rdflib.plugins.sparql import prepareQuery
from rdflib.compare import graph_diff
import threading
from api import inLayer, read_file, full_path, read_schemas, read_extensions, read_examples, namespaces, DataCache, getMasterStore
from apirdflib import getNss
import dev_appserver
dev_appserver.fix_sys_path()

In [6]:
setInTestHarness(True)
import sdoapp
rdflib.plugin.register("json-ld", Serializer, "rdflib_jsonld.serializer", "JsonLDSerializer")


store = getMasterStore()
read_schemas(loadExtensions=True)
read_extensions(sdoapp.ENABLED_EXTENSIONS)
graphs = list(store.graphs())


In [14]:
for g in graphs:
    print g, len(g)


<urn:x-rdflib:default> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']. 0


In [13]:
import pprint
from api import *

read_schemas() # previously automatic

pprint.pprint( "Found %s types." % len( GetAllTypes() ) )

'Found 0 types.'


In [18]:
import os
import pprint

from google.appengine.api import memcache
from google.appengine.api import mail
from google.appengine.api import urlfetch
from google.appengine.ext import db

# pprint.pprint(os.environ.copy())

import pprint
from api import *

read_schemas() # previously automatic

pprint.pprint( "Found %s types." % len( GetAllTypes() ) )

for t in ["Article", "CreativeWork", "Product", "MedicalEntity"]: 
    someType = Unit.GetUnit(t)
    p_subtypes = GetImmediateSubtypes(someType)
    msg =  ( "Direct subtypes of %s: %s" % ( someType.id, ', '.join([str(x.id) for x in p_subtypes]) ) )
    logging.debug(msg) # on cmdline if run as: dev_appserver.py --log_level debug .
    pprint.pprint(msg)

'Found 0 types.'


AttributeError: 'NoneType' object has no attribute 'id'